# Cahn-Hilliard equation

This example is implemented in the Python file <a href="CahnHilliard.py" target="_blank">CahnHilliard.py</a> and it illustrates how to:

- Implement periodic boundary conditions;
- Implement initial conditions with small random perturbation.

Note that this example is based on the FEniCS demo: <a href="https://fenicsproject.org/docs/dolfin/latest/python/demos/cahn-hilliard/demo_cahn-hilliard.py.html" target="_blank">click here</a>.

## Equation and problem definition

The <a href="https://en.wikipedia.org/wiki/Cahn%E2%80%93Hilliard_equation" target="_blank">Cahn-Hilliard equation</a> describes the process of phase separation, by which the two components of a binary fluid spontaneously separate and form domains enriched in one of the two components. The driving force for the phase separation is the tendency of the system to lower the total free energy
$$G[c] = \int_\Omega d{\bf x}\, \left[f(c) + \frac{1}{2}\lambda (\nabla c)^2\right],$$
where $c$ is an order parameter and the two phases correspond to $c=0$ and $c=1$. The first term in the above equation describes the bulk free energy and the second term the interfacial energy between the two phases. The bulk free energy density is commonly described with the double well potential and in this example we use the free energy density $f(c)=100 c^2 (1-c)^2$.

To describe the kinetics of the phase separation process it is convenient to first introduce the chemical potential $\mu$, which can be obtained as the functional derivative of the free energy. More precisely, we calculate the variation of the free energy $G[c+\delta c]=G[c]+\delta G$, where the variation is
$$\delta G= \int_\Omega d{\bf x} \ \left[f'(c) \delta c + \lambda \nabla c \cdot \nabla \delta c\right]=\int_\Omega d{\bf x}\  \left[f'(c) - \lambda (\nabla^2 c) \right] \delta c \equiv \int_\Omega  d{\bf x}\ \mu \delta c,$$
where we used integration by parts and defined the chemical potential
$$\mu = f'(c) - \lambda \nabla^2 c.$$
The gradients of the chemical potential provide the driving force for the redistribution of material, and this kinetics is described with
$$\frac{\partial c}{\partial t} = \nabla \cdot (M \nabla \mu),$$
where $M$ is the mobility.

### Discretizaton of time 

In order to solve the two coupled PDEs for the concentration $c({\bf x},t)$ and chemical potential $\mu({\bf x},t)$, we first discuss the time discretization. Fields are evaluated at discrete timesteps $t_n = n \Delta t$, where $n=0,1,2,\ldots$. We use the notation $c_{n}({\bf x})\equiv c({\bf x},t_n)$ and $\mu_{n}({\bf x})\equiv \mu({\bf x},t_n)$. The two coupled PDEs can then be written as 
    $$\begin{aligned}
    \frac{c_{n+1}-c_{n}}{\Delta t} &= \nabla \cdot (M \nabla \mu_{n+\theta}),\\
    \mu_{n+1} &= f'(c_{n+1}) - \lambda \nabla^2 c_{n+1},\\
    \end{aligned}$$    
where $\mu_{n+\theta} = (1-\theta) \mu_{n} + \theta \mu_{n+1}$. Different choices for the parameter $\theta$ correspond to the forward Euler method ($\theta=0$), the backward Euler method ($\theta=1$), and the Crank–Nicolson method ($\theta=1/2$). Check the <a href="https://en.wikipedia.org/wiki/Crank%E2%80%93Nicolson_method" target="_blank">Wikipedia article</a> for more information about the differences between these three methods. In this example we use the Crank–Nicolson method ($\theta=1/2$). The general procedure is to start with the initial concentrations $c_0({\bf x})$, from which we can calculate the chemical potentials $\mu_0({\bf x})$ at time 0. Then we iteratively calculate the fields ($c_{n+1}$, $\mu_{n+1}$) at time $t_{n+1}$ from the values of fields ($c_{n+1}$, $\mu_{n+1}$) at time $t_n$.

### Weak formulation of the problem

The final step is to write the weak formulation of the coupled PDEs
    $$\begin{aligned}\int_\Omega d{\bf x} \
    \left[\frac{c_{n+1}-c_{n}}{\Delta t} - \nabla \cdot (M \nabla \mu_{n+\theta})\right] q & = 0,\\
    \int_\Omega d{\bf x} \ \left[\mu_{n+1} - f'(c_{n+1}) + \lambda \nabla^2 c_{n+1}\right] v & = 0,\\
    \end{aligned}$$
where we introduced two *test functions* $q$ and $v$. After the integration by parts the two coupled PDEs can be rewritten as
    $$\begin{aligned}\int_\Omega d{\bf x} \
    \left[\frac{c_{n+1}-c_{n}}{\Delta t} q + M \nabla \mu_{n+\theta} \cdot \nabla q\right]  & = 0,\\
    \int_\Omega d{\bf x} \ \left[\mu_{n+1} v  - f'(c_{n+1})v  - \lambda \nabla c_{n+1} \cdot \nabla v\right] & = 0.\\
    \end{aligned}$$
In these example, we solve the Cahn-Hilliard equation on a unit square domain with periodic boundary conditions. The intial concentration values are chosen randomly on the interval $(0.62,0.64)$. Other values are chosen as in the <a href="https://fenicsproject.org/docs/dolfin/latest/python/demos/cahn-hilliard/demo_cahn-hilliard.py.html" target="_blank">FEniCS demo</a>, i.e. $\lambda = 10^{-2}$ and $\Delta t = 5 \times 10^{-6}$.

## Implementation

As usual, in order to use solve this problem we need to import all necessary modules.

In [1]:
from __future__ import print_function
import random
from fenics import *
from dolfin import *
from mshr import *

To implement periodic boundary conditions, we first note that the boundary of the unit square domain can be divided into two subdomains: the unique points on $\Gamma_{inside}$ and the points on $\Gamma_{mapped}$ that can be mapped to the $\Gamma_{inside}$ with periodic boundary conditions. Here we choose the bottom and left boundaries to be in the set $\Gamma_{inside}$ except for the corner points $(1,0)$ and $(0,1)$, which can be mapped to the point (0,0).
<div align="center">    
    <img src="figs/CahnHilliard_domain.png" style="width: 250px;"/>
</div>

To impose periodic boundary conditions in FEniCS, we implement the `SubDomain` class. In this class we have to provide two functions called `inside` and `map`. The `inside` function should return `True` for the unique boundary points on $\Gamma_{inside}$. The `map` function tells how the boundary points on $\Gamma_{mapped}$ are mapped to the points on $\Gamma_{inside}$. 

In [2]:
# Sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    def inside(self, x, on_boundary):
        # return True if on left or bottom boundary AND NOT
        # on one of the two corners (0, 1) and (1, 0)
        return bool((near(x[0], 0) or near(x[1], 0)) and
                (not ((near(x[0], 0) and near(x[1], 1)) or
                        (near(x[0], 1) and near(x[1], 0)))) and on_boundary)

    def map(self, x, y):
        if near(x[0], 1) and near(x[1], 1):
            y[0] = x[0] - 1.
            y[1] = x[1] - 1.
        elif near(x[0], 1):
            y[0] = x[0] - 1.
            y[1] = x[1]
        else:   # near(x[1], 1)
            y[0] = x[0]
            y[1] = x[1] - 1.

Next, we create a mesh for a unit square domain and define the function space, where periodic boundary conditions are imposed with the `constrained_domain` argument. Here we use mixed elements, where the first ellement corresponds to the concentration field $c$ and the second element to the chemical potential field $\mu$. 

In [3]:
# Create mesh and define function space with periodic boundary conditions
mesh = UnitSquareMesh.create(96, 96, CellType.Type.quadrilateral)
P = FiniteElement('Lagrange', mesh.ufl_cell(), 1)
MFS = FunctionSpace(mesh, MixedElement([P,P]),constrained_domain=PeriodicBoundary())

We define functions ${\bf u}_{new}$ and ${\bf u}_{old}$, where we store fields $(c_{n+1},\mu_{n+1})$ and $(c_{n},\mu_{n})$, respectively. We also split the function into the concentration and chemical potential fields. In a similar way we introduce the test function, which is split into two test functions $q$ and $v$.

In [4]:
# Define functions
u_new = Function(MFS)  # solution for the next step
u_old  = Function(MFS)  # solution from previous step
u_new.rename("fields","")
# Split mixed functions
c_new, mu_new  = split(u_new)
c_old, mu_old = split(u_old)

# Define test functions
tf = TestFunction(MFS)
q, v  = split(tf)

To prescribe intitial conditions with the concentration values that are chosen randomly from the interval $(0.62,0.64)$ we implement the `UserExpression` class. In the constructor `(__init__)`, the random number generator is seeded. If the program is run in parallel, the random number generator is seeded using the rank (process number) to ensure a different sequence of numbers on each process. The function `eval` returns the values for a function at a given point $\bf x$. The first component of the function is a randomized value of the concentration $c$ and the second value is the intial value of the chemical potential.
The function `value_shape` declares that the `UseExpression` is a vector with a dimension two. 

In [5]:
# Class representing the intial conditions
class InitialConditions(UserExpression):
    def __init__(self, **kwargs):
        random.seed(2 + MPI.rank(MPI.comm_world))
        super().__init__(**kwargs)
    def eval(self, values, x):
        values[0] = 0.63 + 0.02*(0.5 - random.random()) # concentration
        values[1] = 0.0 # chemical potential
    def value_shape(self):
        return (2,)
    
# Create intial conditions and interpolate
u_init = InitialConditions(degree=1)
u_new.interpolate(u_init)
u_old.interpolate(u_init)    

To calculate the contribution $f'(c)$ for the chemical potential we use automated differentiation. The first line declares that $c$ is a variable that some function can be differentiated with respect to. The next line is the function $f=100 c^2 (1-c)^2$ defined in the problem statement, and the third line performs the differentiation of $f$ with respect to the variable $c$.

In [6]:
# Compute the chemical potential df/dc
c_new = variable(c_new)
f    = 100*c_new**2*(1-c_new)**2
dfdc = diff(f, c_new)

Next we implement the weak form of coupled PDEs
    $$\begin{aligned}\int_\Omega d{\bf x} \
    \left[\frac{c_{n+1}-c_{n}}{\Delta t} q + M \nabla \mu_{n+\theta} \cdot \nabla q\right]  & = 0,\\
    \int_\Omega d{\bf x} \ \left[\mu_{n+1} v  - f'(c_{n+1})v  - \lambda \nabla c_{n+1} \cdot \nabla v\right] & = 0.\\
    \end{aligned}$$

In [7]:
lmbda  = 1.0e-02  # surface parameter
dt     = 5.0e-06  # time step
theta  = 0.5      # time stepping family, e.g. theta=1 -> backward Euler, theta=0.5 -> Crank-Nicolson

# mu_(n+theta)
mu_mid = (1.0-theta)*mu_old + theta*mu_new

# Weak statement of the equations
Res_0 = (c_new - c_old)/dt*q*dx + dot(grad(mu_mid), grad(q))*dx
Res_1 = mu_new*v*dx - dfdc*v*dx - lmbda*dot(grad(c_new), grad(v))*dx
Res = Res_0 + Res_1

Prepare files, where we will store concentration and chemical potential fields for the visualization in <a href="https://www.paraview.org/" target="_blank">ParaView</a>.

In [8]:
# Output files for concentration and chemical potential
fileC = File("data/concentration.pvd", "compressed")
fileM = File("data/chem_potential.pvd", "compressed")

Iteratively solve the coupled PDEs for 50 time steps. At each step we first copy the values of fields from the ${\bf u}_{new}$ function to the ${\bf u}_{old}$ function. Then we solve the coupled PDEs and store the new values of fields in the function ${\bf u}_{new}$. Finally, we save the values of fields for visualization in <a href="https://www.paraview.org/" target="_blank">ParaView</a> 

In [9]:
# Step in time
t = 0.0
T = 50*dt
while (t < T):
    t += dt
    print(t)
    u_old.vector()[:] = u_new.vector()
    solve(Res == 0, u_new)
    fileC << (u_new.split()[0], t)
    fileM << (u_new.split()[1], t)

5e-06
1e-05
1.5000000000000002e-05
2e-05
2.5e-05
3e-05
3.5000000000000004e-05
4e-05
4.5e-05
5e-05
5.5e-05
6e-05
6.500000000000001e-05
7.000000000000001e-05
7.500000000000001e-05
8e-05
8.5e-05
9e-05
9.5e-05
0.0001
0.000105
0.00011
0.000115
0.00012
0.000125
0.00013000000000000002
0.00013500000000000003
0.00014000000000000004
0.00014500000000000006
0.00015000000000000007
0.00015500000000000008
0.0001600000000000001
0.0001650000000000001
0.00017000000000000012
0.00017500000000000013
0.00018000000000000015
0.00018500000000000016
0.00019000000000000017
0.00019500000000000019
0.0002000000000000002
0.0002050000000000002
0.00021000000000000023
0.00021500000000000024
0.00022000000000000025
0.00022500000000000026
0.00023000000000000028
0.0002350000000000003
0.0002400000000000003
0.0002450000000000003
0.00025000000000000033


## Complete code

In [10]:
from __future__ import print_function
import random
from fenics import *
from dolfin import *
from mshr import *

    
# Sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    def inside(self, x, on_boundary):
        # return True if on left or bottom boundary AND NOT
        # on one of the two corners (0, 1) and (1, 0)
        return bool((near(x[0], 0) or near(x[1], 0)) and
                (not ((near(x[0], 0) and near(x[1], 1)) or
                        (near(x[0], 1) and near(x[1], 0)))) and on_boundary)

    def map(self, x, y):
        if near(x[0], 1) and near(x[1], 1):
            y[0] = x[0] - 1.
            y[1] = x[1] - 1.
        elif near(x[0], 1):
            y[0] = x[0] - 1.
            y[1] = x[1]
        else:   # near(x[1], 1)
            y[0] = x[0]
            y[1] = x[1] - 1.
            


# Create mesh and define function space with periodic boundary conditions
mesh = UnitSquareMesh.create(96, 96, CellType.Type.quadrilateral)
P = FiniteElement('Lagrange', mesh.ufl_cell(), 1)
MFS = FunctionSpace(mesh, MixedElement([P,P]),constrained_domain=PeriodicBoundary())

# Define functions
u_new = Function(MFS)  # solution for the next step
u_old  = Function(MFS)  # solution from previous step
u_new.rename("fields","")
# Split mixed functions
c_new, mu_new  = split(u_new)
c_old, mu_old = split(u_old)

# Define test functions
tf = TestFunction(MFS)
q, v  = split(tf)

# Define test functions
tf = TestFunction(MFS)
q, v  = split(tf)


# Class representing the intial conditions
class InitialConditions(UserExpression):
    def __init__(self, **kwargs):
        random.seed(2 + MPI.rank(MPI.comm_world))
        super().__init__(**kwargs)
    def eval(self, values, x):
        values[0] = 0.63 + 0.02*(0.5 - random.random()) # concentration
        values[1] = 0.0 # chemical potential
    def value_shape(self):
        return (2,)
    

# Create intial conditions and interpolate
u_init = InitialConditions(degree=1)
u_new.interpolate(u_init)
u_old.interpolate(u_init)



# Compute the chemical potential df/dc
c_new = variable(c_new)
f    = 100*c_new**2*(1-c_new)**2
dfdc = diff(f, c_new)


lmbda  = 1.0e-02  # surface parameter
dt     = 5.0e-06  # time step
theta  = 0.5      # time stepping family, e.g. theta=1 -> backward Euler, theta=0.5 -> Crank-Nicolson

# mu_(n+theta)
mu_mid = (1.0-theta)*mu_old + theta*mu_new

# Weak statement of the equations
Res_0 = (c_new - c_old)/dt*q*dx + dot(grad(mu_mid), grad(q))*dx
Res_1 = mu_new*v*dx - dfdc*v*dx - lmbda*dot(grad(c_new), grad(v))*dx
Res = Res_0 + Res_1


# Output files for concentration and chemical potential
fileC = File("data/concentration.pvd", "compressed")
fileM = File("data/chem_potential.pvd", "compressed")


# Step in time
t = 0.0
T = 50*dt
while (t < T):
    t += dt
    print(t)
    u_old.vector()[:] = u_new.vector()
    solve(Res == 0, u_new)
    fileC << (u_new.split()[0], t)
    fileM << (u_new.split()[1], t)


5e-06
1e-05
1.5000000000000002e-05
2e-05
2.5e-05
3e-05
3.5000000000000004e-05
4e-05
4.5e-05
5e-05
5.5e-05
6e-05
6.500000000000001e-05
7.000000000000001e-05
7.500000000000001e-05
8e-05
8.5e-05
9e-05
9.5e-05
0.0001
0.000105
0.00011
0.000115
0.00012
0.000125
0.00013000000000000002
0.00013500000000000003
0.00014000000000000004
0.00014500000000000006
0.00015000000000000007
0.00015500000000000008
0.0001600000000000001
0.0001650000000000001
0.00017000000000000012
0.00017500000000000013
0.00018000000000000015
0.00018500000000000016
0.00019000000000000017
0.00019500000000000019
0.0002000000000000002
0.0002050000000000002
0.00021000000000000023
0.00021500000000000024
0.00022000000000000025
0.00022500000000000026
0.00023000000000000028
0.0002350000000000003
0.0002400000000000003
0.0002450000000000003
0.00025000000000000033
